# Tracing directors' career journeys using time series data
*Julie Nguyen, PhD candidate in Management (Organizational Behavior), McGill University*

Welcome to another chapter of our exploration into the world of social networks and their impact on on movie directors' careers. So far, we've identified our sample of directors (`Phase_1_Tracking_Movie_Directors_Career.ipynb`), mapped the collaboration networks within the film industry and calculated the annual social capital of every creative worker (`Phase_2_Constructing_Filmmaker_Network.ipynb`), and predicted the genders of our directors (`Phase_3_Predicting_Director_Gender.ipynb`).

**What are we aiming to do?**

Now, we shift our focus to constructing a detailed time series dataset. This dataset will trace the career journey of each director year by year, from their debut until the present year, 2023. This setup allows us to understand and predict various patterns like career lengths or peaks and declines in productivity. 

**How do we do this?**

1. Constructing time series data: We'll begin by creating a dataset where each row represents a director's career in a given year, from the year their first film was released to 2023. 

2. Incorporating outcome variables: We will integrate two crucial variables into our dataset:
- Annual directorial engagement: This variable indicates whether a director has actively directed a movie in a given year. It serves as a primary indicator of whether they are still active in the industry.
- Dropout indicator: This variable indicates whether a director has potentially ceased directing, defined as having not directed any movie in the last 10 years. This variable identifies the point at which a director may have stepped away from their directing career.

3. Incorporating survival analysis variables: To further enrich our analysis, we'll include `start_time` and `stop_time` for each director's career relative to their debut. These are key for survival analysis, which will allow us to study how long directors remain active and the timing of any career transitions. By basing these times on each director’s debut, we focus on the duration and phases of their careers without being constrained by specific calendar years.

**Looking ahead**

By the end of this notebook, we’ll have developed a comprehensive dataset that tracks each director's career progression year-by-year, along with indicators that reflect their ongoing involvement and longevity in the film industry. This preparation sets the stage for our forthcoming analyses, where we'll examine how various factors—such as network connections and gender—may influence the trajectories of these careers.

# Constructing a time series dataset 

In this part of our analysis, we construct the `directors_years` DataFrame. This is a structured timeline that captures the career trajectory of film directors on an annual basis, from their debut year up to the year 2023. Each row in the data represents a single year in the career of a director. Columns include:
- `nconst_director`: A unique identifier for each director.
- `year`: The specific year being considered for that director's career.
- `debut_year`: The year when the director made their debut.

**Source data**

For this task, we use the `directors_full_filmography` dataset, which we created in a previous notebook (`Phase_1_Tracking_Movie_Directors_Career.ipynb`). This dataset catalogs all films directed by 63,169 directors in our study, with each row associating a director (`nconst`) with a movie (`tconst`) they made.

**Defining the study period**

Since our sample of directors made their debut between 2003 and 2013, our career tracking will span from 2003 to 2023. This period is specifically chosen to ensure that we have at least ten years of follow-up data after each director's debut, facilitating a robust long-term analysis of career trajectories.

**Creating a year-by-year framework**

To effectively track each director's activity:
- We first generate all possible combinations of directors and years within our defined range. This exhaustive approach ensures we capture every year of a director's career from their debut to 2023, even if they did not release any movies in some years. 
- Next, for each director we'll exclude years before they began directing. This way, we don't waste time looking at years before they started their directing career.

In [1]:
# Importing necessary libraries for data manipulation and handling
import pandas as pd  # data manipulation
import os  # interacting with the operating system, such as file paths

# Set the working directory to where the project files are located
os.chdir('/Users/mac/Library/CloudStorage/OneDrive-McGillUniversity/Work/Projects/Gender and brokerage/WomenLeaders_SocialNetworks')

In [2]:
# Load the filmography dataset for movie directors (2003-2023)
directors_full_filmography = pd.read_csv('directors_full_filmography.csv')

# Display the initial rows of the dataset to understand its structure
directors_full_filmography.head()

,tconst,startYear,genres,nconst,firstYear,averageRating,numVotes
0,tt0108549,2004.0,"Comedy,Mystery",nm1131265,2004.0,7.8,34.0
1,tt0108549,2004.0,"Comedy,Mystery",nm1130611,2004.0,7.8,34.0
2,tt0117461,2003.0,"Comedy,Romance",nm0290651,2003.0,6.3,24.0
3,tt0117743,2008.0,"Drama,Romance",nm0404033,2003.0,6.7,64.0
4,tt0118141,2005.0,Drama,nm0000417,2005.0,5.3,950.0


In [3]:
# Convert the 'startYear' column to integer values 
directors_full_filmography['startYear'] = directors_full_filmography['startYear'].astype(int)

# Identify unique directors and define the range of years for our analysis (2003-2023)
# This range is chosen based on the debut years of directors being studied (2003-2013), extending to 2023 for a long-term look at their careers
directors = directors_full_filmography['nconst'].unique()
years = range(2003, 2024) 

# Generate all possible director-year combinations within the specified range
# This creates a framework to examine each director's career on a yearly basis
directors_years = pd.DataFrame([(director, year) for director in directors for year in years], columns=['nconst', 'year'])

# Merge the generated combinations with the original filmography data to include each director's debut year
# This allows us to filter combinations to only include years after each director's debut
directors_years = directors_years.merge(directors_full_filmography[['nconst', 'firstYear']].drop_duplicates(), on='nconst', how='left')

# Filter out rows that represent years before a director's debut, ensuring the dataset only contains relevant director-year pairs
directors_years = directors_years[directors_years['year'] >= directors_years['firstYear']].drop(columns=['firstYear'])
directors_years.rename(columns={'nconst': 'nconst_director'}, inplace=True)

# Identify the debut year for each director
debut_years = directors_years.groupby('nconst_director')['year'].min().reset_index()
debut_years.rename(columns={'year': 'debut_year'}, inplace=True)

# Merge the debut year back into the directors_years dataset to add a debut_year column
directors_years = pd.merge(directors_years, debut_years, on='nconst_director', how='left')

In [4]:
directors_years.head(30)

,nconst_director,year,debut_year
0,nm1131265,2004,2004
1,nm1131265,2005,2004
2,nm1131265,2006,2004
3,nm1131265,2007,2004
4,nm1131265,2008,2004
5,nm1131265,2009,2004
6,nm1131265,2010,2004
7,nm1131265,2011,2004
8,nm1131265,2012,2004
9,nm1131265,2013,2004


Everything is in order! Now we can move on to adding variables indicating career longevity (our outcome variables) to our time series dataset.

# Creating outcome variables for time series data

In this segment of the analysis, we'll create two crucial outcome variables that will help us measure each director's career trajectory and add them to the `directors_years` dataset:

1. Active Engagement (`made_movie`): This binary indicator specifies whether a director released at least one movie in a given year. This provides a direct measure of a director's active engagement in directing during each year of their career. Notably, for a director's debut year, we set this indicator to zero, ensuring that our analysis focuses on activity after their debut.

2. Career Continuity (`dropout`): This variable indicates whether a director potentially dropped out of directing in a given year. We determine this by identifying the last year in which directors released a movie. Directors who have not released any movies since 2013 (the last decade of the observation period) are considered to have potentially dropped out. Conversely, for directors who were active after 2013, we do not specify a dropout year, implying continued involvement in the industry.

As we gear up for survival analysis, let's also create 

3. Relative Career Timing (`start_time` and `stop_time`): These variables are essential for survival analysis, as they allow us to measure the duration of directors' active phases. We calculate these timing variables relative to each director's debut year. This approach allows us to standardize the career timeline across our sample, focusing on the progression and duration of directors' careers without the variability introduced by different debut years.

By the end, we should have a dataset where each row represents a single year in the career of a director, with the following columns:
- `nconst_director`: A unique identifier for each director.
- `year`: The specific year being considered for that director's career.
- `debut_year`: The year when the director made their debut.
- `num_movies`: The number of movies released by the director in that particular year.
- `made_movie`: A binary indicator stating whether the director released any movies in that year (1 if yes, 0 if no).
- `dropout`: Indicates whether the director is considered to have potentially dropped out of the industry by that year.
- `start_time` and `stop_time`: The relative start and stop times from each director's debut year.

Let's start with creating a binary indicator (`made_movie`) to identify whether a director was active in a given year.

In [5]:
# taking a look at the director filmography data to figure out how to create outcome variables
directors_full_filmography

,tconst,startYear,genres,nconst,firstYear,averageRating,numVotes
0,tt0108549,2004,"Comedy,Mystery",nm1131265,2004.0,7.8,34.0
1,tt0108549,2004,"Comedy,Mystery",nm1130611,2004.0,7.8,34.0
2,tt0117461,2003,"Comedy,Romance",nm0290651,2003.0,6.3,24.0
3,tt0117743,2008,"Drama,Romance",nm0404033,2003.0,6.7,64.0
4,tt0118141,2005,Drama,nm0000417,2005.0,5.3,950.0
...,...,...,...,...,...,...,...
128529,tt9916362,2020,"Drama,History",nm1893148,2008.0,6.4,5687.0
128530,tt9916538,2019,Drama,nm4457074,2011.0,8.6,7.0
128531,tt9916622,2015,Documentary,nm9272490,2012.0,NaN,NaN
128532,tt9916754,2013,Documentary,nm9272490,2012.0,NaN,NaN


In [6]:
# Aggregate the director's filmography data to count unique movies released by each director per year.
movies_released_per_year = directors_full_filmography.groupby(['nconst', 'startYear'])['tconst'].nunique().reset_index()

# Rename columns for clarity: director ID as 'nconst_director', release year as 'year', and number of movies as 'num_movies'.
movies_released_per_year.rename(columns={'nconst': 'nconst_director', 'startYear': 'year', 'tconst': 'num_movies'}, inplace=True)

# Merge the yearly movie release counts with the main dataset to include the number of movies each director released each year.
directors_years = directors_years.merge(movies_released_per_year, on=['nconst_director', 'year'], how='left')

# Replace missing values in 'num_movies' with 0, indicating no movies were released by the director in those years.
directors_years['num_movies'] = directors_years['num_movies'].fillna(0)

# Create a binary indicator 'made_movie' to denote whether the director released any movies in a given year (1 if yes, 0 if no).
directors_years['made_movie'] = (directors_years['num_movies'] > 0).astype(int)

# For the director's debut year, set 'num_movies' and 'made_movie' to 0 so that the variables reflect activity after debut
directors_years.loc[directors_years['year'] == directors_years['debut_year'], ['num_movies', 'made_movie']] = 0

In [7]:
directors_years.head(25)

,nconst_director,year,debut_year,num_movies,made_movie
0,nm1131265,2004,2004,0.0,0
1,nm1131265,2005,2004,0.0,0
2,nm1131265,2006,2004,0.0,0
3,nm1131265,2007,2004,0.0,0
4,nm1131265,2008,2004,0.0,0
5,nm1131265,2009,2004,0.0,0
6,nm1131265,2010,2004,0.0,0
7,nm1131265,2011,2004,0.0,0
8,nm1131265,2012,2004,0.0,0
9,nm1131265,2013,2004,1.0,1


Looks great! Next, we'll move on to identifying career dropout (`dropout`).

In [8]:
# Group by director ID and identify the latest year of movie release to determine the last active year for each director.
last_movie_year = directors_full_filmography.groupby('nconst')['startYear'].max().reset_index()

# Identify directors who haven't made any movies between 2013 and 2023, who are considered to have potentially dropped out
dropout_directors = last_movie_year[last_movie_year['startYear'] < 2013].copy()

# Set 'dropoutYear' as the year following their last movie release, marking when they might have left the industry.
dropout_directors['dropoutYear'] = dropout_directors['startYear'] + 1

# For directors active after 2013, set 'dropoutYear' as NA, indicating they haven't dropped out within the study period.
active_directors = last_movie_year[last_movie_year['startYear'] >= 2013].copy()
active_directors['dropoutYear'] = pd.NA

# Combine the dataframes to get a complete view of all directors' potential dropout years.
all_directors_dropout = pd.concat([dropout_directors, active_directors], ignore_index=True).sort_values(by='nconst')

# Display the first and last few rows of the combined dataframe
all_directors_dropout

,nconst,startYear,dropoutYear
36527,nm0000083,2022,<NA>
0,nm0000136,2009,2010
1,nm0000147,2010,2011
36528,nm0000154,2016,<NA>
36529,nm0000155,2013,<NA>
...,...,...,...
63168,nm9986224,2021,<NA>
36523,nm9990558,2009,2010
36524,nm9990640,2007,2008
36525,nm9990734,2009,2010


In [9]:
# Merge the 'dropoutYear' information into the main dataset to indicate the potential dropout year for each director.
directors_years = pd.merge(directors_years, all_directors_dropout[['nconst', 'dropoutYear']],
                           how='left', left_on='nconst_director', right_on='nconst')

# Remove the redundant 'nconst' column after merging.
directors_years.drop(columns='nconst', inplace=True)

# Initialize a 'dropout' column with 0, indicating no dropout event by default.
directors_years['dropout'] = 0

# Update 'dropout' to 1 for years equal to or beyond the 'dropoutYear', marking it as the potential dropout event.
directors_years['dropout'] = ((directors_years['year'] >= directors_years['dropoutYear'].fillna(9999)).astype(int))

/var/folders/1j/csv118695x53qkzhd9p2tgcc0000gn/T/ipykernel_3461/608082436.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  directors_years['dropout'] = ((directors_years['year'] >= directors_years['dropoutYear'].fillna(9999)).astype(int))


In [10]:
# Display the first few rows
directors_years.head(30)

,nconst_director,year,debut_year,num_movies,made_movie,dropoutYear,dropout
0,nm1131265,2004,2004,0.0,0,<NA>,0
1,nm1131265,2005,2004,0.0,0,<NA>,0
2,nm1131265,2006,2004,0.0,0,<NA>,0
3,nm1131265,2007,2004,0.0,0,<NA>,0
4,nm1131265,2008,2004,0.0,0,<NA>,0
5,nm1131265,2009,2004,0.0,0,<NA>,0
6,nm1131265,2010,2004,0.0,0,<NA>,0
7,nm1131265,2011,2004,0.0,0,<NA>,0
8,nm1131265,2012,2004,0.0,0,<NA>,0
9,nm1131265,2013,2004,1.0,1,<NA>,0


Fianlly, we create `start_time` and `stop_time` variables for each director.

In [11]:
# Calculate relative start and stop times based on the director's debut year
directors_years['start_time'] = directors_years['year'] - directors_years['debut_year']
directors_years['stop_time'] = directors_years['start_time'] + 1

In [12]:
directors_years.head(30)

,nconst_director,year,debut_year,num_movies,made_movie,dropoutYear,dropout,start_time,stop_time
0,nm1131265,2004,2004,0.0,0,<NA>,0,0,1
1,nm1131265,2005,2004,0.0,0,<NA>,0,1,2
2,nm1131265,2006,2004,0.0,0,<NA>,0,2,3
3,nm1131265,2007,2004,0.0,0,<NA>,0,3,4
4,nm1131265,2008,2004,0.0,0,<NA>,0,4,5
5,nm1131265,2009,2004,0.0,0,<NA>,0,5,6
6,nm1131265,2010,2004,0.0,0,<NA>,0,6,7
7,nm1131265,2011,2004,0.0,0,<NA>,0,7,8
8,nm1131265,2012,2004,0.0,0,<NA>,0,8,9
9,nm1131265,2013,2004,1.0,1,<NA>,0,9,10


In [13]:
directors_years.to_csv("directors_years.csv", index=False)

# Creating cross-sectional dataset

Let's also create a cross-sectional dataset with information on the first 11 years of each director's career, starting from their debut. The dataset is designed to investigate the time it takes for a director to release their second movie using survival analysis. It will include the following columns:
- `nconst_director`: A unique identifier for each director. 
- `debut_year`: The year in which a director released their first movie. 
- `time_to_second_movie`: The number of years from the director's debut to the release of their second movie. This is the primary variable of interest for survival analysis, indicating how quickly directors follow up their debut with another film. If a director has not released a second movie within the first 11 years, this is filled with an upper limit value (e.g., 11), representing censored data in survival analysis.
- `event`: A binary indicator coded as 1 if the second movie was made (non-censored), and 0 if the second movie was not made by the end of the study period (censored). This variable is essential for survival analysis, as it helps differentiate between directors who have reached the event of interest (second movie release) and those who have not within the study frame.
- `total_movies`: The total number of movies a director released within the first 11 years of their career.

In [49]:
# Filter the data to include only the first 11 years of each director's career
filtered_data = directors_years[directors_years['start_time'] <= 10].copy()

# Calculate cumulative movies for each director over time
filtered_data['cumulative_movies'] = filtered_data.groupby('nconst_director')['num_movies'].cumsum()

# Determine the minimum year when cumulative movies reach or exceed 1 
# (meaning their second movie since we set 0 for number of movies in their debut year to focus on analyzing career after debut)
second_movie_time = filtered_data[filtered_data['cumulative_movies'] >= 1].groupby('nconst_director')['start_time'].min()

# Create a DataFrame from the second_movie_time series
second_movie_df = second_movie_time.reset_index()
second_movie_df.columns = ['nconst_director', 'time_to_second_movie']

# Retrieve all director IDs and their debut year
# Merge this with the second_movie_df to include all directors
# Directors without a second movie will have NaN in 'time_to_second_movie'
directors_cross_sectional = filtered_data[['nconst_director', 'debut_year']].drop_duplicates().merge(second_movie_df, on='nconst_director', how='left')

# Create event indicator
directors_cross_sectional['event'] = (~directors_cross_sectional['time_to_second_movie'].isna()).astype(int)  # 1 if second movie made, 0 if not

# Handling the time variable, using the maximum study period for those who didn't make a second movie:
directors_cross_sectional['time_to_second_movie'] = directors_cross_sectional['time_to_second_movie'].fillna(11)

In [50]:
directors_cross_sectional

,nconst_director,debut_year,time_to_second_movie,event
0,nm1131265,2004,9.0,1
1,nm1130611,2004,11.0,0
2,nm0290651,2003,11.0,0
3,nm0404033,2003,5.0,1
4,nm0000417,2005,2.0,1
...,...,...,...,...
63164,nm4394847,2012,11.0,0
63165,nm10527671,2012,11.0,0
63166,nm10532602,2012,11.0,0
63167,nm4453202,2012,11.0,0


In [53]:
# Calculate the total number of movies each director made within the first 11 years
total_movies = directors_years[directors_years['start_time'] < 11].groupby('nconst_director')['num_movies'].sum().reset_index()
total_movies.columns = ['nconst_director', 'total_movies']

# Merge this back to the main DataFrame to include all directors
directors_cross_sectional = directors_cross_sectional.merge(total_movies, on='nconst_director', how='left')

In [54]:
directors_cross_sectional

,nconst_director,debut_year,time_to_second_movie,event,total_movies
0,nm1131265,2004,9.0,1,1.0
1,nm1130611,2004,11.0,0,0.0
2,nm0290651,2003,11.0,0,0.0
3,nm0404033,2003,5.0,1,1.0
4,nm0000417,2005,2.0,1,1.0
...,...,...,...,...,...
63164,nm4394847,2012,11.0,0,0.0
63165,nm10527671,2012,11.0,0,0.0
63166,nm10532602,2012,11.0,0,0.0
63167,nm4453202,2012,11.0,0,0.0


In [55]:
directors_cross_sectional.to_csv("directors_cross_sectional.csv", index=False)